In [4]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings  # or any embedding model of your choice
from langchain.vectorstores import FAISS


# data_dir = "./dataset"


# loader = DirectoryLoader(data_dir, glob="**/*.txt", show_progress=True)
# documents = loader.load()

# for doc in documents[:2]:
#     print(doc.metadata)


In [5]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Path to your research papers folder
data_dir = "./dataset"

# Use a glob pattern that matches PDF files and set the loader class to PyPDFLoader.
loader = DirectoryLoader(
    data_dir,
    glob="**/*.pdf",         # Match PDF files
    loader_cls=PyPDFLoader,    # Use the PDF loader
    show_progress=True
)

# Load the documents
documents = loader.load()

 88%|████████████████████████████████████████████████████████████████████████▎         | 15/17 [00:06<00:00,  2.33it/s]


In [6]:

for doc in documents:
  print(doc.metadata["source"])
  

dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R001.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R002.pdf
dataset\Non-Publishable\R003.pdf
dataset\Non-Publishable\R003.pdf
dataset\Non-Publishable\R003.pdf
dataset\Non-Publishable\R003.pdf
dataset\No

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
# for i in docs:#just checking the split bitches
#     print(i)
    
#     print("------------------------------------------------------------------")


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "all-MiniLM-L6-v2"  
embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [9]:
vectorstore = FAISS.from_documents(docs, embeddings)


In [10]:
import pprint as p
from langchain.document_loaders import PyPDFLoader

def classify_paper(file_path):
    # Check the file extension and load content accordingly.
    if file_path.lower().endswith(".pdf"):
        # Use PyPDFLoader to load the PDF file.
        loader = PyPDFLoader(file_path)
        # The loader returns a list of Document objects.
        documents = loader.load()
        # Concatenate the content from all pages/documents.
        content = "\n".join(doc.page_content for doc in documents)
    else:
        # Assume it's a text file.
        with open(file_path, 'r') as f:
            content = f.read()
    
    # Build the query: include instructions and list of known organizations.
   
    query = content

    # Run the chain – the retriever will add similar examples as context.
  
    return query

# Example usage:
new_paper_path = r"C:\Users\avnex\Downloads\3534678.3539321.pdf"
query = classify_paper(new_paper_path)

#########################################################################################################################################
query_embedding = embeddings.embed_query(query)  # Convert query to vector
similar_docs = vectorstore.similarity_search(query, k=3)  # Retrieve top 3 similar docs

for doc in similar_docs:
    
    # print(doc.page_content)
    print(doc.metadata["source"])
    print("----------------------------------------------------------------")
# Create a list of formatted strings (page content + source)
# formatted_docs = [f"data: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in similar_docs]
# final_output = "\n\n".join(formatted_docs)
# print(final_output)  




dataset\Publishable\KDD\R010.pdf
----------------------------------------------------------------
dataset\Publishable\KDD\R010.pdf
----------------------------------------------------------------
dataset\Publishable\EMNLP\R008.pdf
----------------------------------------------------------------


In [76]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
llm = OllamaLLM(model="deepseek-r1:7b")



In [79]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Load the LLM
llm = OllamaLLM(model="deepseek-r1:7b")

# Combine retrieved documents into a single string with sources
retrieved_texts = [
    f"Data: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in similar_docs
]
retrieved_context = "\n\n".join(retrieved_texts)

# Define the evaluation prompt
evaluation_prompt = PromptTemplate(
    input_variables=["context"],
    template=(
       "you are a research paper analyst tasked with detereming wether a given research paper is publishable based on relevant dataset"
        "the dataset consist of multiple context each containing data and methodological results and sources. the paper is categorized"
        "under publishable and non-publishable based on standard acacdemic criteria.given the following three research paper snippiet anlyse"
        "them carefully and determine whihc one is the most apt for publication.provide your final decision by selecting the most suitable research paper if none of them meets publishable standard state them expliciy"
    )
)

# Create the LLM chain
llm_chain = LLMChain(llm=llm, prompt=evaluation_prompt)

# Run the classification
result = llm_chain.invoke({"context": retrieved_context})

# Extract and print only the final classification result
print("Final Classification:", result["text"].strip())


Final Classification: {'context': 'Data: Learning sub-intervals for pooling is found to be crucial for activity recognition.\nMethod RGB Flow Two-stream\nRandom 16.3 16.3 16.3\nInceptionV3 + mean-pool 35.6 47.2 45.3\nInceptionV3 + max-pool 47.9 48.6 54.4\nInceptionV3 + pyramid 49.7 53.2 55.3\nInceptionV3 + LSTM 47.6 55.6 57.7\nInceptionV3 + temporal conv 47.2 55.2 56.1\nInceptionV3 + sub-events 56.2 62.5 62.6\nI3D + mean-pool 42.4 47.6 52.7\nI3D + max-pool 48.3 53.4 57.2\nI3D + pyramid 53.2 56.7 58.7\nI3D + LSTM 48.2 53.1 53.1\nI3D + temporal conv 52.8 57.1 58.4\nI3D + sub-events 55.5 61.2 61.3\nTable 5 shows the average precision for each activity class. Learning temporal structure is particularly\nbeneficial for frame-based features (e.g., InceptionV3), which capture less temporal information\n5\nSource: dataset\\Publishable\\CVPR\\R006.pdf\n\nData: Next, the team selects keyframes. From the set {IDi}n\ni=1, keyframes {IK\nj }k\nj=1 ⊆ {IDi}n\ni=1 are\nchosen. A method is implemented 

In [81]:
# Extract and print only the final classification result
print("Final Classification:", result["text"].strip())


Final Classification: <think>
Okay, so I'm trying to figure out which of these three research papers is the most suitable for publication based on academic criteria. Let me go through each one step by step.

Starting with Research Paper A: It's about the impact of renewable energy policies on carbon emissions reduction. The study uses panel data from 2015 to 2020, which seems like a good time frame. They used fixed effects and difference-in-differences models—those are solid econometric methods. However, I notice they only applied these to one country, which might limit the generalizability of their findings. Also, there's nothing about replicating the results or checking for robustness with other models like random effects or maybe even machine learning techniques. Maybe adding a control group from another region could strengthen it.

Moving on to Research Paper B: This one looks at the effect of social media on mental health using a dataset from 2010-2023. They used OLS regression an